In [16]:
import os
import configparser
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from IPython.display import display, Latex
from warnings import filterwarnings
filterwarnings('ignore')

In [13]:
API_KEY = 'key'

In [14]:
# Define the embedding model
embeddings = MistralAIEmbeddings(model="mistral-embed", mistral_api_key=API_KEY)

/home/andrey/miniconda3/lib/python3.12/site-packages/langchain_mistralai/embeddings.py:180: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [34]:
for i in range(1, 12):
    loader = TextLoader(f'kons/{i}.tex')
    docs = loader.load() 
    text_splitter = RecursiveCharacterTextSplitter()
    documents = text_splitter.split_documents(docs)

    try:
        if not os.path.exists(f'vesctors/faiss_index_{i}'):
            vector = FAISS.from_documents(documents, embeddings)
            vector.save_local(f"vesctors/faiss_index_{i}")
    except:
        print(i)


vector = FAISS.load_local("vesctors/faiss_index_1", embeddings,  allow_dangerous_deserialization=True)

for i in range(2, 12):
    tmp = FAISS.load_local(f"vesctors/faiss_index_{i}", embeddings,  allow_dangerous_deserialization=True)
    vector.merge_from(tmp)
vector.save_local(f"vesctors")

An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429
An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


7
8


An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


9


In [35]:
vector = FAISS.load_local("vesctors", embeddings,  allow_dangerous_deserialization=True)

In [36]:
retriever = vector.as_retriever()

In [37]:
model = ChatMistralAI(mistral_api_key=API_KEY)

In [38]:
prompt = ChatPromptTemplate.from_template("""Ответь на следующий вопрос основываясь только на предоставленном контексте:


<контекст>
{context}
</контекст>

Вопрос: {input}""")

# Работа RAG на основе консультаций

In [43]:
# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": '''Равномерная сходимость функциональных последовательностей
и рядов. Непрерывность, интегрируемость и дифференцируемость
суммы функционального ряда.'''})
display(Latex(response['answer']))

<IPython.core.display.Latex object>

# Работа обычной модели

In [45]:
from langchain_core.messages import HumanMessage
messages = [HumanMessage(content='''Равномерная сходимость функциональных последовательностей
и рядов. Непрерывность, интегрируемость и дифференцируемость
суммы функционального ряда.''')]
display(Latex(model.invoke(messages).content))

<IPython.core.display.Latex object>